In [1]:
# !pip install tensorflow
#!pip install transformers
#!pip install tf-keras
# !pip install imblearn

In [2]:
import sys
from pathlib import Path

# Zum übergeordneten Ordner 'src' wechseln
current_path = Path().resolve()
src_path = current_path.parents[2]  # Zwei Stufen nach oben

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

src_path


WindowsPath('C:/Users/Nasiba/Documents/1 Master Data Science/Projektpraktikum/WebScience24')

In [3]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from collections import Counter
#nlp = spacy.load("en_core_web_sm")
from nltk import word_tokenize
pd.set_option('display.max_colwidth', None)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from vector_functions import vectorize_word2vec,  vectorize_glove_new,vectorize_glove, vectorize_glove_test_data,  vectorize_word2vec_test_data,  vectorize_glove_test_data_predict, vectorize_fasttext_2, vectorize_fasttext_test_data
import nltk
from tensorflow.keras.models import load_model

import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import tokenizer_from_json



#### Beladung der Daten

In [5]:

source_filepath_name = os.path.abspath(os.path.join(src_path, 'data/mixed_dataset/test.csv'))

df_test = pd.read_csv(source_filepath_name, encoding='utf-8', index_col=0)


source_filepath_name_train = os.path.abspath(os.path.join(src_path, 'data/mixed_dataset/train_cleaned_rnn.csv'))

df_cleaned = pd.read_csv(source_filepath_name_train, encoding='utf-8', index_col=0)




### Vektorisierung

#### GloVe 

In [6]:


twitter_glove_path = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/glove.6B.200d.txt'))


X_train_vectors, X_test_vectors, y_train, y_test, glove_embeddings,tokenizer = vectorize_glove(df=df_cleaned, text_column ="tweet_cleaned", label_column="label", glove_path=twitter_glove_path)




c:\Users\Nasiba\Documents\1 Master Data Science\Projektpraktikum\WebScience24\src\3. Deep Learning Approach\nasiba\vector_functions.py:532: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df[text_column].astype(str)


In [8]:


# Tokenizer in JSON speichern
tokenizer_path = save_path_model = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/tokenizer/tokenizer.json'))
tokenizer_json = tokenizer.to_json()
with open(tokenizer_path, "w", encoding="utf-8") as f:
    f.write(tokenizer_json)

print(f"Tokenizer wurde gespeichert unter: {tokenizer_path}")


Tokenizer wurde gespeichert unter: C:\Users\Nasiba\Documents\1 Master Data Science\Projektpraktikum\WebScience24\src\3. Deep Learning Approach\nasiba\tokenizer\tokenizer.json


In [12]:


# Gespeicherten Tokenizer laden
with open(tokenizer_path, "r", encoding="utf-8") as f:
    tokenizer_data = f.read()
    tokenizer_loaded = tokenizer_from_json(tokenizer_data)


In [13]:

source_filepath_name = os.path.abspath(os.path.join(src_path, 'data/mixed_dataset/test.csv'))

df_test = pd.read_csv(source_filepath_name, encoding='utf-8', index_col=0)

X_test_data_vectors_3, text_col_test_data, glove_embeddings_3_test, tokenizer_3_test = vectorize_glove_test_data_predict(df=df_test, text_column ="tweet", glove_path=twitter_glove_path, tokenizer=tokenizer_loaded)

tokenizer_3_test

In [14]:

save_path_model = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/models/model_3.keras'))
model_3 = load_model(save_path_model)
model_3 

<Sequential name=sequential, built=True>

In [15]:
predictions_3_test= model_3.predict(X_test_data_vectors_3)
threshold=0.35 # Oder teste 0.4 oder 0.6
predicted_classes_3_test = (predictions_3_test > threshold).astype(int)
df_predictions_3 = df_test[['tweet']].copy()  # Textspalte übernehmen
df_predictions_3['predicted_label'] =predicted_classes_3_test  # Vorhersagen hinzufügen

df_positive_predictions_3 = df_predictions_3[df_predictions_3['predicted_label'] == 1][['tweet', 'predicted_label']]

df_positive_predictions_3

1019/1019 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step


,tweet,predicted_label
11885,nothing would be more dangerous than to give #putinschoice the impression we accept #stopracism #boycott?â¦,1
6640,and #misogyny is alive and well on the #left. never let them tell you it isn't.,1
23680,give hope. dismantle systemic racism. #alliesforblacklives #happyholidays #blacklivesmatters #dismantle â¦,1
31311,happy bihday california gurls ð ð­ð¬ð¦ð©ðªð«ð­ð¬ð¦ð©ðªð« #myfavoritevideo #video #california #gurls â¦,1
23597,hiv infection from oral sex white girl has sex with black guy,1
...,...,...
176325,"RT @user: The way Tricksnipers act is just ugh. 89% of them are Chief Keef, Soulja Boy niggers. Absolute sewage waste",1
176328,Cool. Fuck your racist ass you cunt,1
176344,@user Certainly the paper shows that the idea that drinking water was the only or main health issue in Iraq is a lie.,1
176366,@user photoshop my nig,1


Model_21 auf testdaten predicten

In [ ]:

save_path_model = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/models/model_17.keras'))
model_17 = load_model(save_path_model)
predictions_17_test = model_17.predict(X_test_data_vectors_3)
threshold=0.35 # Oder teste 0.4 oder 0.6
predicted_classes_17_test = (predictions_17_test > threshold).astype(int)
df_predictions_17 = df_test[['tweet']].copy()  # Textspalte übernehmen
df_predictions_17['predicted_label'] =predicted_classes_17_test  # Vorhersagen hinzufügen

df_positive_predictions_17 = df_predictions_17[df_predictions_17['predicted_label'] == 1][['tweet', 'predicted_label']]

df_positive_predictions_17

1019/1019 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step


,tweet,predicted_label
11885,nothing would be more dangerous than to give #putinschoice the impression we accept #stopracism #boycott?â¦,1
27378,attacks 11 week old baby. kills mother. #shocking #disgusted,1
6640,and #misogyny is alive and well on the #left. never let them tell you it isn't.,1
19019,what is a #detoxdiet? #altwaystoheal! #healthy !,1
12084,politics is great in theory but politicians are all self obsessed point scoring children &amp; idiots take their word as gospel = dangerous,1
...,...,...
176344,@user Certainly the paper shows that the idea that drinking water was the only or main health issue in Iraq is a lie.,1
176360,Even liberals are humiliated by Charlie Rangel&#8217;s claim the GOP backs slavery - Hot Air http://t.co/I4QawESnud,1
176366,@user photoshop my nig,1
176384,And mad hoes you can ask Beavis I get nothing butthead,1


Modell speichern

In [23]:
save_path_labeled_data_3 = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/labeled_data/test_labeled_lstm_md_17.csv'))


df_predictions_17.to_csv(save_path_labeled_data_3, index=False)

In [4]:
save_path_labeled_data_17 = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/labeled_data/test_labeled_lstm_md_17.csv'))

save_path_labeled_data_21 = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/labeled_data/test_labeled_lstm_md_21.csv'))

save_path_labeled_data_3 = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/labeled_data/test_labeled_lstm_md_3.csv'))

df_17 = pd.read_csv(save_path_labeled_data_17, encoding='utf-8', index_col=0)

df_21 = pd.read_csv(save_path_labeled_data_21, encoding='utf-8', index_col=0)

df_3 = pd.read_csv(save_path_labeled_data_3, encoding='utf-8', index_col=0)

In [12]:
df_17_label0= df_17[df_17['predicted_label']==0].head(100)
df_17_label0.head(50)

,predicted_label
tweet,
@user #cinemaaawards final rehearsals!! gearing up for the evening!! #butterflies #stage ! hope u all like it,0
this was amazing. the weather was not. #musical #london #matilda #westend #weekend â¦,0
child attacked by alligator at walt disney world #waltdisneyworld,0
me rn cause school is over soon,0
is it #good to #sleep when ? #good #sleep,0
hi @user we hear you're coming to mk . where about a will you be?,0
#cloudlovers attack bull game 3d: do you really think that his head was empty around the city. each side i,0
@user #paintnite pay with @user lola_shwgrl and sisters and friends. lifeâ¦,0
i am thankful for computers. #thankful #positive,0


In [11]:
df_17_label1= df_17[df_17['predicted_label']==1].head(200)
df_17_label1

,predicted_label
tweet,
nothing would be more dangerous than to give #putinschoice the impression we accept #stopracism #boycott?â¦,1
attacks 11 week old baby. kills mother. #shocking #disgusted,1
and #misogyny is alive and well on the #left. never let them tell you it isn't.,1
what is a #detoxdiet? #altwaystoheal! #healthy !,1
politics is great in theory but politicians are all self obsessed point scoring children &amp; idiots take their word as gospel = dangerous,1
...,...
@user @user sweetie...that's it? your reto to his message is to critique his 2nd language skills??? speaks volumes ð,1
"@user ms's own sharpton doesn't represent the will of ppl, only his will to gain attention.",1
so the @user tells all the other players there might be a penalty but they're not sure?,1


In [20]:
df_combined = pd.concat([df_17_label0, df_17_label1], ignore_index=False)
df_combined

,predicted_label
tweet,
@user #cinemaaawards final rehearsals!! gearing up for the evening!! #butterflies #stage ! hope u all like it,0
this was amazing. the weather was not. #musical #london #matilda #westend #weekend â¦,0
child attacked by alligator at walt disney world #waltdisneyworld,0
me rn cause school is over soon,0
is it #good to #sleep when ? #good #sleep,0
...,...
@user @user sweetie...that's it? your reto to his message is to critique his 2nd language skills??? speaks volumes ð,1
"@user ms's own sharpton doesn't represent the will of ppl, only his will to gain attention.",1
so the @user tells all the other players there might be a penalty but they're not sure?,1


In [24]:






save_path_labeled_data_manual = os.path.abspath(os.path.join(src_path, 'src/3. Deep Learning Approach/nasiba/labeled_data/test_labeled_manual.csv'))
df_combined.to_csv(save_path_labeled_data_manual, sep=",", encoding="utf-8")

In [18]:
df_3_label1= df_3[df_3['predicted_label']==1]
df_3_label1

,predicted_label
tweet,
nothing would be more dangerous than to give #putinschoice the impression we accept #stopracism #boycott?â¦,1
and #misogyny is alive and well on the #left. never let them tell you it isn't.,1
give hope. dismantle systemic racism. #alliesforblacklives #happyholidays #blacklivesmatters #dismantle â¦,1
happy bihday california gurls ð ð­ð¬ð¦ð©ðªð«ð­ð¬ð¦ð©ðªð« #myfavoritevideo #video #california #gurls â¦,1
hiv infection from oral sex white girl has sex with black guy,1
...,...
"RT @user: The way Tricksnipers act is just ugh. 89% of them are Chief Keef, Soulja Boy niggers. Absolute sewage waste",1
Cool. Fuck your racist ass you cunt,1
@user Certainly the paper shows that the idea that drinking water was the only or main health issue in Iraq is a lie.,1
